In [25]:
# Import necessary libraries
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer

from gensim import corpora
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel

# Download NLTK Resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [32]:
"""
Group Members:

Putri Qistina (SW01081178)
Alyaa Natasha (SW01081000)

"""

'\nGroup Members:\n\nPutri Qistina (SW01081178)\nAlyaa Natasha (SW01081000)\n\n'

In [26]:
data = pd.read_csv('news_dataset.csv')

In [27]:
# Use only the text column
documents = data['text'].dropna().tolist()

# Text Pre-Processing

In [28]:
# remove stopwords, lemmatization, and stemming
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token.isalnum() and not token.isdigit()] # Remove non-alphanumeric tokens and numbers
    tokens = [token for token in tokens if token not in stop_words] # Remove stopwords
    tokens = [stemmer.stem(token) for token in tokens] # Apply stemming
    tokens = [lemmatizer.lemmatize(token) for token in tokens] # Apply lemmatization
    return tokens

# Preprocess every document in the list
preprocessed_documents = [preprocess_text(doc) for doc in documents]

# Document-term Matrix

In [29]:
# Create a Gensim Dictionary object 
dictionary = corpora.Dictionary(preprocessed_documents)

# Convert each preprocessed document into a bag-of-words representation 
corpus = [dictionary.doc2bow(doc) for doc in preprocessed_documents]

# LDA

In [30]:
# Train an LDA model on the corpus with 4 topics using Gensim's LdaModel class
lda_model = LdaModel(corpus, num_topics=4, id2word=dictionary, passes=15)

# Calculate the coherence score 
coherence_model_lda = CoherenceModel(model=lda_model, texts=preprocessed_documents, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()

# Result

In [31]:
# Empty list to store dominant topic labels for each document
article_labels = []

# Iterate over each processed document
for i, doc in enumerate(preprocessed_documents):
   
    bow = dictionary.doc2bow(doc)
    topics = lda_model.get_document_topics(bow)
    dominant_topic = max(topics, key=lambda x: x[1])[0]
    article_labels.append(dominant_topic)
    
# Create DataFrame
df = pd.DataFrame({"The Article": documents, "The Topic": article_labels})

# Print the DataFrame
print("Table with the Articles and Topic:")
print(df)
print()

Table with the Articles and Topic:
                                             The Article  The Topic
0      I was wondering if anyone out there could enli...          3
1      I recently posted an article asking what kind ...          3
2      \nIt depends on your priorities.  A lot of peo...          3
3      an excellent automatic can be found in the sub...          3
4      : Ford and his automobile.  I need information...          3
...                                                  ...        ...
11091  Secrecy in Clipper Chip\n\nThe serial number o...          1
11092  Hi !\n\nI am interested in the source of FEAL ...          1
11093  The actual algorithm is classified, however, t...          1
11094  \n\tThis appears to be generic calling upon th...          3
11095  \nProbably keep quiet and take it, lest they g...          3

[11096 rows x 2 columns]



In [ ]:
# Print the top terms for every topic
print("Top Terms for Each Topic:")
for idx, topic in lda_model.print_topics():
    print(f"Topic {idx}:")
    terms = [term.strip() for term in topic.split("+")]
    for term in terms:
        weight, word = term.split("*")
        print(f"-{word.strip()} (weight: {weight.strip()})")
    print()

# Display the coherence score
print(f'Topic Coherence Score (C_V): {coherence_lda:.4f}')